###  Create a response table using qgrid to pick the data and pivot table to format

In [ ]:
import numpy as np
import pandas as pd
import os
import gcsfs
import xarray as xr

In [ ]:
import pandas as pd

import qgrid
#dfcat = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv')
dfcat = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores.csv')

len(dfcat)

In [ ]:
table_id = 'day'
exps = ['historical','ssp126','ssp245','ssp370']
variables = ['pr','tasmin','tasmax']
models = 'all'
#members = 'r1i1p1f1'

In [ ]:
# define a simple search on keywords
def search_df(df, verbose= False, **search):
    "search by keywords - if list, then match exactly, otherwise match as substring"
    keys = ['activity_id','institution_id','source_id','experiment_id','member_id', 'table_id', 'variable_id', 'grid_label']
    d = df
    for skey in search.keys():
        
        if isinstance(search[skey], str):  # match a string as a substring
            d = d[d[skey].str.contains(search[skey])]
        else:
            dk = []
            for key in search[skey]:       # match a list of strings exactly
                dk += [d[d[skey]==key]]
            d = pd.concat(dk)
            keys.remove(skey)
    if verbose:
        for key in keys:
            print(key,' = ',list(d[key].unique()))      
    return d

In [ ]:
if models == 'all':
    dfs = search_df(dfcat,variable_id=variables,experiment_id=exps,table_id=table_id,member_id=members)
else:
    dfs = search_df(dfcat,variable_id=variables,experiment_id=exps,table_id=table_id,source_id=models)

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# use this to get number of models (one per model)
dm = dfs[['experiment_id','source_id','member_id','variable_id','grid_label']].groupby([
#         'experiment_id','variable_id']).nunique()[['source_id']]
# use this to get models*members
         'experiment_id','member_id','source_id','variable_id']).nunique()[['grid_label']]

table = pd.DataFrame.pivot_table(dm,
                                 values='grid_label',
                                 index=['source_id','variable_id','member_id'],
                                 columns=['experiment_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)
print('activity_drs, table_id = ',df.activity_id.unique(),df.table_id.unique())
print(table)

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

dm = dfs[['experiment_id','source_id','member_id','variable_id']].groupby([
         'experiment_id','source_id','variable_id']).nunique()[['source_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='source_id',
                                 index=['variable_id'],
                                 columns=['experiment_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)
print('activity_id, table_id = ',df.activity_id.unique(),df.table_id.unique())
print(table)